# Import

In [2]:
from cobra.io import read_sbml_model
import os
import nbimporter
from 02_balance_metabolic_models import extract_met_info_model


SyntaxError: invalid syntax (595107004.py, line 4)

In [2]:
models_path = "Models/charge_balance"
models_curation = {}
for model_name in (f for f in os.listdir(models_path) if f.endswith(".xml")):
    model = read_sbml_model(f"{models_path}/{model_name}")
    model.solver = "cplex"
    name = str(model_name[:3]+"_curate")
    models_curation[name] = model

models_curation = {key: models_curation[key] for key in sorted(models_curation.keys())}  # sorts the dictionary alphabetically
AA1_curate, AA2_curate, AA3_curate, AA4_curate, AA5_curate, AA6_curate, AA7_curate = [models_curation[f"AA{i}_curate"] for i in range(1, 8)]

Restricted license - for non-production use only - expires 2026-11-23


In [3]:
def get_rxn(model, rxn_id, print_mass = False):
    rxn = model.reactions.get_by_id(rxn_id)
    charges = {met.id: met.charge for met in rxn.metabolites}
    masses = {met.id: met.formula for met in rxn.metabolites}
    if print_mass:
        print(rxn, charges, masses)
    print(rxn, charges)

# Check Metabolite Duplicates
Based on the Memote reports that were done after the first mass balancing of the models (i.e. after using Frowins Scripts), we need to check the identified duplicate metabolites. Are these real duplicated metabolites?

Copy possible duplicated metabolites from the memote reports (report_AAx_balanced.html)

In [102]:
AA1_met_dup = [["3hbcoa_c","3hbycoa_c"],["aacald_c","amacald_c"],["dd2coa_c","trans_dd2coa_c"],["nh3_c","nh4_c"],["orn__L_c","orn_c"],["orn__L_e","orn_e"]]

In [103]:
AA2_duplicates = [["1btol_c","btoh_c"],["2h3opp_c","2h3oppan_c"],["34dhpacet_c","34dhpha_c"],["3hbcoa_c","3hbycoa_c"],["aacald_c","amacald_c"],["abt__L_c","abt_c"],["dd2coa_c","trans_dd2coa_c"],["metox_c","metsox_S__L_c"],["nh3_c","nh4_c"],["orn__L_c","orn_c"]]

In [104]:
AA3_duplicates = [["3hbcoa_c","3hbycoa_c"],["abt__L_c","abt_c"],["abt__L_e","abt_e"],["dd2coa_c","trans_dd2coa_c"],["metox_c","metsox_S__L_c"],["nh3_c","nh4_c"],["orn__L_c","orn_c"],["orn__L_e","orn_e"],["stcoa_c","strcoa_c"]]

In [106]:
AA4_duplicates = [["2kmb_c","4met2obut_c"],["34dhpacet_c","34dhpha_c"],["metox_c","metsox_S__L_c"],["metox_e","metsox_S__L_e"],["nh3_c","nh4_c"]]

In [105]:
AA5_duplicates = [["1btol_c","btoh_c"],["3hbcoa_c","3hbycoa_c"],["abt__L_c","abt_c"],["dd2coa_c","trans_dd2coa_c"],["metox_c","metsox_S__L_c"],["nh3_c","nh4_c"],["orn__L_c","orn_c"]]

In [107]:
AA6_duplicates = [["1btol_c","btoh_c"],["2h3opp_c","2h3oppan_c"],["2kmb_c","4met2obut_c"],["34dhpacet_c","34dhpha_c"],["3hbcoa_c","3hbycoa_c"],["5d4dglcr_c","5dh4dglc_c"],["aacald_c","amacald_c"],["dd2coa_c","trans_dd2coa_c"],["eths_c","ethso3_c"],["eths_e","ethso3_e"],["galct__D_c","galctr__D_c"],["galct__D_e","galctr__D_e"],["glcn__D_c","glcn_c"],["isetac_c","istnt_c"],["isetac_e","istnt_e"],["metox_c","metsox_S__L_c"],["nh3_c","nh4_c"],["orn__L_c","orn_c"],["orn__L_e","orn_e"],["sula_c","sulfac_c"],["sula_e","sulfac_e"]]


In [108]:
AA7_duplicates = [["aacald_c","amacald_c"],["eths_c","ethso3_c"],["eths_e","ethso3_e"],["isetac_c","istnt_c"],["isetac_e","istnt_e"],["nh3_c","nh4_c"],["sula_c","sulfac_c"],["sula_e","sulfac_e"]]


In [109]:
duplicates = [val for name, val in globals().items() if isinstance(val, list) and name.startswith('AA')]

In [22]:
# get the unique duplicates across all models
unique_duplicates = set()
for outer in duplicates:
    for inner in outer:
        unique_duplicates.add(tuple(inner))

In [26]:
print(f'There are {len(unique_duplicates)} duplicates (tuples) across all models, i.e. in total {len(unique_duplicates)*2} metabolites to check.')

There are 25 duplicates (tuples) across all models, i.e. in total 50 metabolites to check.


In [ ]:
ecoli = read_sbml_model("iML1515.xml")
ecoli.solver = 'cplex'

In [3]:
import cobra
from cobra.io import read_sbml_model, write_sbml_model
# from macaw.main import run_all_tests

ecoli = read_sbml_model('iML1515.xml')

Restricted license - for non-production use only - expires 2026-11-23


# Check Reaction Duplicates

In [4]:
def check_delete(model, compare_list):
    if len(compare_list) != 2:
        print("Please provide exactly two reactions to compare")
        return

    if compare_list[0] in model.reactions and compare_list[1] in model.reactions:
        rxn1 = model.reactions.get_by_id(compare_list[0])
        rxn2 = model.reactions.get_by_id(compare_list[1])

        if rxn1.gene_reaction_rule == rxn2.gene_reaction_rule:
            print(f"Same GPR for both reactions: {rxn1.gene_reaction_rule}")
        else:
            print(f"different GPR for both reactions: {rxn1.gene_reaction_rule} for {compare_list[0]} vs. {rxn2.gene_reaction_rule} for {compare_list[1]}")

In [5]:
model_mets = {f"AA{i}_mets": extract_met_info_model(models_curation[f"AA{i}_curate"]) for i in range(1, 8)}

In [13]:
model_mets["AA1_mets"]

,bigg_id,model_id,model_formula,model_charge
0,10fthf,10fthf_c,C20H21N7O7,-2
1,12dgr120,12dgr120_c,C27H52O5,0
2,12dgr120,12dgr120_p,C27H52O5,0
3,12dgr140,12dgr140_c,C31H60O5,0
4,12dgr140,12dgr140_p,C31H60O5,0
...,...,...,...,...
1526,xylb,xylb_e,C10H18O9,0
1527,xylu__D,xylu__D_c,C5H10O5,0
1528,zn2,zn2_c,Zn,2
1529,zn2,zn2_e,Zn,2


In [3]:
AA1_react_dup = [["EX_orn__L_e","EX_orn_e"],["APPAT","PTPATi"],["ARGDI","ARGDr"],["CYSS","CYSS_2"],["ECOAH1","ECOAH1_1"],["ECOAH5","ECOAH5_1"],["GLYK","GLYK1"],["HACD1","HACD1_1"],["HMGL","HMGL_2"],["HPPK2","HPPK2_1"],["IDPh_1","PPA"],["MCCC","MCTC_1"],["MEPCT","MEPCT_1"],["NTP1","ATPM"],["OGDE1","OXOADLR"],["PPRGL","PRAGSr"]]

In [120]:
aa = [["EX_eths_e","EX_ethso3_e"],["EX_isetac_e","EX_istnt_e"],["EX_sula_e","EX_sulfac_e"],["4CMLCL_kt","CMLDC"],["APPAT","PTPATi"],["ARGDI","ARGDr"],["BUPN","UPPN"],["GTPDPK","GTPDPK_1"],["HDH","HISTD"],["HMGL","HMGL_2"],["IDPh_1","PPA"],["IG3PS","IG3PS_1"],["MEPCT","MEPCT_1"],["NTP1","ATPM"]]


In [ ]:
# BEFORE deletion we always have to check if the other reaction is also in a model
to_delete = ["PTPATi", "ARGDr", "CYSS_2", "GLYK1", "HPPK2_1", "IDPh_1", "MCTC_1", "MEPCT_1", "NTP1", "OGDE1", "PPRGL", "ALCD2x"]
# maybe: ECOAH5_1, ECOAH1_1,HACD1_1

In [8]:
check_delete(AA2_curate, ["PRFGS","PRFGS_1"])

different GPR for both reactions: WP_007875279_1 or (WP_079211777_1 and WP_079211778_1 and WP_079211779_1) for PRFGS vs. (WP_079211777_1 and WP_079211778_1) or (WP_079211777_1 and WP_079211778_1 and WP_079211779_1) for PRFGS_1


In [3]:
check_delete(AA2_curate, ["ASPA2","3SALATAi"])

NameError: name 'AA2_curate' is not defined

In [96]:
get_rxn(AA1_curate, "OGDE1")

OGDE1: 2oxoadp_c + h_c + lpam_c --> S_gtrdhdlp_c + co2_c {'2oxoadp_c': -2, 'lpam_c': 0, 'h_c': 1, 'S_gtrdhdlp_c': -1, 'co2_c': 0}


In [95]:
get_rxn(AA1_curate, "OXOADLR")

OXOADLR: 2oxoadp_c + h_c + lpam_c --> S_gtrdhdlp_c + co2_c {'2oxoadp_c': -2, 'h_c': 1, 'lpam_c': 0, 'S_gtrdhdlp_c': -1, 'co2_c': 0}


In [55]:
for rxn in AA1_curate.reactions:
    if "ACACT" in rxn.name or "ACACT" in rxn.id:
        print(rxn)


ACACT10: 3oddcoa_c + coa_c --> accoa_c + dcacoa_c
ACACT11: 3ohcoa_c + coa_c --> accoa_c + btcoa_c
ACACT12: 3ohdcoa_c + coa_c --> accoa_c + tdcoa_c
ACACT13: 3odcoa_c + coa_c --> accoa_c + occoa_c
ACACT1r: 2.0 accoa_c <=> aacoa_c + coa_c
ACACT2r: accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
ACACT3r: accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
ACACT4r: accoa_c + occoa_c <=> 3odcoa_c + coa_c
ACACT5r: accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
ACACT5r_1: accoa_c + dccoa_c <=> 3oddcoa_c + coa_c
ACACT6r_1: accoa_c + ddcoa_c <=> 3otdcoa_c + coa_c
ACACT7r: accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c
ACACT8: 3oocoa_c + coa_c --> accoa_c + hxcoa_c
ACACT8r: 3ohodcoa_c + coa_c <=> accoa_c + pmtcoa_c
ACACT9: 3otdcoa_c + coa_c --> accoa_c + ddcacoa_c


In [8]:
print("orn_e" in AA1_met_dup)

False


# Check Loops